# Analyze data using ilastik

The notebook shows how load images contained in a dataset from OMERO as numpy arrays and analyze them in ilastik.
It assumes that ilastik project is linked to the dataset. The 5D-numpy array is in the order expected by the ilastik project. The order might need to be adjusted depending on the ilastik project.

### Insert required packages

In [9]:
import numpy
import os
import subprocess

import tempfile
import omero.clients
from omero.gateway import BlitzGateway
from getpass import getpass

### Create a connection to an OMERO server

In [10]:
HOST = 'outreach.openmicroscopy.org'
PORT = 4064
conn = BlitzGateway(raw_input("Username: "), getpass("OMERO Password: "), host=HOST, port=PORT)
conn.connect()

Username: idr
OMERO Password: ········


False

### Enter the dataset ID

In [6]:
dataset_id = 8252

### Helper function: Load the ilastik project linked to the dataset

In [ ]:
def load_model(dataset_id, path):
    dataset = conn.getObject("Dataset", dataset_id)
    # Go through all the annotations on the Dataset
    for ann in dataset.listAnnotations():
        if isinstance(ann, omero.gateway.FileAnnotationWrapper):
            name = ann.getFile().getName()
            #Select the ilatisk project TODO: use namespace
            if name.endswith(".ilp"):
                file_path = os.path.join(path, name)
                with open(str(file_path), 'w') as f:
                    for chunk in ann.getFileInChunks():
                        f.write(chunk)
                return file_path"

### Helper function: load an Image as 5D-numpy array: order TZYXC

In [7]:
def load_numpy_array(image, tmp_file):
    pixels = image.getPrimaryPixels()
    size_z = image.getSizeZ()
    size_c = image.getSizeC()
    size_t = image.getSizeT()
    size_y = image.getSizeY()
    size_x = image.getSizeX()
    z, t, c = 0, 0, 0 # first plane of the image
    
    zct_list = []
    for t in range(size_t):
        for z in range(size_z): # get the Z-stack
            for c in range(size_c): # all channels
                zct_list.append((z, c, t))
                
    l = []
    # Load all the planes as YX numpy array
    planes = pixels.getPlanes(zct_list)
    j = 0
    k = 0
    tmp_c = []
    tmp_z = []
    print("z:%s t:%s c:%s y:%s x:%s" % (size_z, size_t, size_c, size_y, size_x))
    # axis tzyxc
    print("Downloading image %s" % image.getName())
    for i, p in enumerate(planes):
        if k < size_z:
            if j < size_c:
                tmp_c.append(p)
                j = j + 1
            if j == size_c:
                tmp_z.append(numpy.dstack(tmp_c)) # use dstack to have c at the end
                tmp_c = []
                j = 0
                k = k + 1
        if k == size_z: # done with the stack
            l.append(numpy.stack(tmp_z))
            tmp_z = []
            k = 0
    
    all_planes = numpy.stack(l)
    numpy.save(tmp_file, all_planes)
    print("Image converted")
    return tmp_file

In [ ]:
def plane_gen():
    """
    Set up a generator of 2D numpy arrays.
    
    The createImage method below expects planes in the order specified here
    (for z.. for c.. for t..)
    
    """
    
    for z in range(data.shape[0]): # all Z sections data.shape[0]
        for c in range(data.shape[1]): # all channels
            for t in range(data.shape[2]): # all time-points
                yield data[z][c][t]

### Load each image as an 5D-numpy array and analyze. Save the probabilities as an OMERO image

In [ ]:
# Create a directory where to upload the file
path = tempfile.mkdtemp()
if not os.path.exists(path):
    os.makedirs(path)
    
images = conn.getObjects('Image', opts={'dataset': dataset_id})
# Load the model linked to the dataset
model_file = load_model(dataset_id, path)

# Create a new dataset where to upload the generated images
dataset_obj = omero.model.DatasetI()
dataset_obj.setName(omero.rtypes.rstring(("ilastik_probabilities_%s" % dataset_id)))
dataset_obj = conn.getUpdateService().saveAndReturnObject(dataset_obj)

for image in images:
    filename, file_extension = os.path.splitext(image.getName())
    tmp_file = os.path.join(path, filename +".npy")
    tmp_file = load_numpy_array(image, tmp_file)
    
    # run ilastik headless
    print('running ilastik headless using %s on file %s' % (model_file, image.getName()))
    # save output in zctyx order expected by OMERO
    cmd = ["/opt/python-apps/ilastik-release/run_ilastik.sh",
           '--headless',
           '--project=%s' % model_file,
           '--output_format=numpy',
           '--output_axis_order=zctyx', tmp_file]
    
    try:
        subprocess.check_call(cmd)
    except subprocess.CalledProcessError as e:
        print(e.output)
    
    # Save the probabilities file to the image
    print("Saving Probabilities as an Image in OMERO")
    name = filename +"_Probabilities"
    probabilities = os.path.join(path, name +".npy")
    data = numpy.load(probabilities)
    desc = "ilastik probabilities results"
    conn.createImageFromNumpySeq(plane_gen(), name, data.shape[0], data.shape[1], data.shape[2], description=desc, dataset=dataset_obj)

print("done")

### Close the connection to OMERO server

In [ ]:
conn.close()